## SED fitting using a functional form SFH;

This notebook will demosntrate how we get SED results with broadband data points using a function form for SFHs.


In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import ascii,fits
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.table import QTable
import astropy.units as u

from astropy import __version__ as asver
asver

In [ ]:
# https://github.com/mtakahiro/gsf/tree/version1.4
import gsf
print(gsf.__version__)

from gsf.function import get_input
from gsf.gsf import run_gsf_template
from gsf.plot_sed_logA import plot_sed_tau, plot_corner_physparam_frame, plot_corner_physparam_summary
from gsf.plot_sfh_logA import plot_sfh_tau

### Setup gsf

In [ ]:
# Initial setup for gsf.

# Data directory;
DIR_DATA = './data/'


# Auto load input dictionary;
inputs = get_input()

# change Z;
# Flag;
fplt = 0
inputs['DIR_TEMP'] = './templates_del/'

# Output directory;
inputs['DIR_OUT'] = './output_del/'

# If templates exit already, then let's save time.
# (But if you changed metallicity range or age pixels, fplt needs to be 0.)
if os.path.exists('%s/spec_all.asdf'%inputs['DIR_TEMP']):
    fplt = 1

inputs['ID'] = '32707'

# Redshift will be obtained from BB catalog;
inputs['ZGAL'] = 1.647

# Redshift as a free parameter?
inputs['ZMC'] = 0

# Metallicity range, in logZsun;
inputs['ZMIN'] = -0.4
inputs['ZMAX'] = 0.41
inputs['DELZ'] = 0.2
# You can fix metallicity;
#inputs['ZFIX'] = 0.0

# Tau model specific;
inputs['SFH_FORM'] = 4 # delayed tau.
inputs['NPEAK'] = 2 # Number of SFH components.
inputs['AGEMIN'] = -1
inputs['AGEMAX'] = 1.2
inputs['DELAGE'] = 0.1
inputs['TAUMIN'] = -1.
inputs['TAUMAX'] = 1.0
inputs['DELTAU'] = 0.1
inputs['NIMF'] = 1

# Templates;
# Currently in prep.
inputs['BPASS'] = 0

# Data;
inputs['DIR_FILT'] = './filter/'
inputs['CAT_BB'] = DIR_DATA + 'uds_PHOT_xy.cat'

# Filters;
# Each number corresponds to EAZY's filter ids. See also filter/filt_Sep20.lst
# These numbers need to be found in inputs['CAT_BB'] file.
inputs['FILTER'] = '205,88,122,79,236,123,124,239,125,203,263,204,264,265,18,19,20,21'

# MCMC part;
inputs['NMC'] = 10000 # NMC for the main SED fit
inputs['NMCZ'] = 30 # NMC for the redshift fit

# Visual inspection;
# Set to 0 (False), as Notebook cannot show actively iterating plot;
inputs['ZVIS'] = 0

# Initial fit:
inputs['FNELD'] = 0


In [ ]:
fplt
inputs

In [ ]:
# Then, run template generate function;
mb = run_gsf_template(inputs, fplt=fplt)
fplt = 1


In [ ]:
# You can write down the input file in an ascii file.
from gsf.function import write_input
write_input(inputs, file_out='gsf_del.input')


### Now, run the whole SED fitting;

In [ ]:
mb = run_gsf_template(inputs, fplt=10)


# If updated any parameter in inputs, run this; 
if False:
    mb.update_input(inputs)

# Since already z-fit done, we can skip z-fit;
skip_fitz = True

# Main;
flag_suc = mb.main(cornerplot=True, specplot=1, sigz=1.0, ezmin=0.01, ferr=0, 
                   f_move=False, skip_fitz=skip_fitz, f_shuffle=True)

# If the call above fail, try again with f_shuffle=False.


In [ ]:
# Plot SFH;

# Plot Starforming Main Sequence from Speagle+14?
f_SFMS = True
f_symbol = True
skip_zhist = True
tau_lim = 0.01
tset_SFR_SED = 10
mmax = 300

plot_sfh_tau(mb, f_comp=mb.ftaucomp, fil_path=mb.DIR_FILT, mmax=mmax,
inputs=mb.inputs, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_silence=True, 
f_SFMS=f_SFMS, f_symbol=f_symbol, skip_zhist=skip_zhist, tau_lim=tau_lim, tset_SFR_SED=tset_SFR_SED)


In [ ]:
# Plot SED;
plot_sed_tau(mb, fil_path=mb.DIR_FILT,
figpdf=False, save_sed=True, inputs=mb.inputs, mmax=300,
f_fill=True, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_label=True)
